In [63]:
import numpy as np
import os
import cv2
import torch

In [64]:
test = torch.load("data/test_dataset.pt")

RuntimeError: PytorchStreamReader failed locating file data.pkl: file not found

In [ ]:
imgs, labels = test.data, test.labels

In [ ]:
import numpy as np

pairs = []

# Convert lists to numpy arrays for faster processing
imgs = np.array(imgs)
labels = np.array(labels)

for img, label in zip(imgs, labels):
    # Get indices of images with the same label and different labels
    same_class_indices = np.where(labels == label)[0]
    different_class_indices = np.where(labels != label)[0]

    if np.random.random() < 0.5:
        # Same class pair
        pair_index = np.random.choice(same_class_indices)
    else:
        # Different class pair
        pair_index = np.random.choice(different_class_indices)

    pair_img = imgs[pair_index]
    pair_label = labels[pair_index]
    
    if np.array(pair_img).shape != (3, 64, 64):
        print("Tf")
    
    pairs.append((img, pair_img, label))


In [ ]:
from torch.utils.data import Dataset

class OpenSet(Dataset):
    def __init__(self, pairs, labels):
        self.pairs = pairs
        self.labels = labels

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, index):
        try:
            return self.pairs[index], self.labels[index]
        except IndexError as e:
            print(e)
            print(len(self.labels), index)

    def get_imgs(self):
        return self.pairs

    def get_n_classes(self):
        return len(set(self.labels))

    def labels_to_long(self):
        self.labels = self.labels.astype(np.int64)

    def get_class_data(self, label):
        return [self.pairs[i] for i, l in enumerate(self.labels) if l == label]


In [ ]:
ps = [[pair[0], pair[1]] for pair in pairs]
ls = [pair[2] for pair in pairs]
test_ds = OpenSet(ps, ls)

In [ ]:
torch.save(test_ds, "data/_test_dataset.pt")

MemoryError: 